In [9]:
import pandas as pd
import json
from pprint import pprint
import numpy as np
from os import listdir
from os.path import isfile, join

In [17]:
def get_data(filename:str):
    
    with open(filename) as f:
        json_file = json.load(f)
    
    data = json_file['people'][0]['pose_keypoints_2d']
    assert len(data) == 75
    return data

def pipeline(path, label):
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    df = pd.DataFrame(columns=list(range(75)))
    for i, file in enumerate(onlyfiles):
        df.loc[i] = get_data(path+file)
    df['label'] = label
    return df
    

good_data = pipeline('warrior/', label=1)
bad_data = pipeline('warrior_incorrect/', label=0)
#get_data('warrior/warrior_000000000039_keypoints.json', classification = 'good')
    

In [20]:
good_data.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,label
0,664.680,204.282,0.918086,692.073,259.115,0.875232,646.980,257.194,0.832140,572.597,...,515.760,609.804,0.599547,521.606,603.885,0.594741,576.510,601.873,0.755656,1
1,664.665,204.246,0.918837,692.090,259.068,0.875573,646.958,257.168,0.822406,572.624,...,515.737,611.718,0.592743,521.600,603.895,0.596312,576.498,601.908,0.757170,1
2,664.684,206.193,0.931179,693.983,257.119,0.875670,647.023,255.218,0.807673,572.645,...,517.692,611.715,0.589091,521.646,603.913,0.586224,578.445,601.916,0.748488,1
3,666.623,212.076,0.964515,696.038,255.150,0.897790,649.037,253.210,0.800950,574.538,...,515.785,609.764,0.605959,521.614,603.875,0.596971,578.452,601.928,0.742447,1
4,666.646,212.079,0.957779,696.036,255.137,0.896309,649.010,251.283,0.797011,574.496,...,515.775,609.786,0.600176,521.619,603.891,0.589176,578.461,601.957,0.746312,1


In [22]:
full_data = pd.concat([good_data,bad_data])
full_data

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,label
0,664.680,204.282,0.918086,692.073,259.115,0.875232,646.980,257.194,0.832140,572.597,...,515.760,609.804,0.599547,521.606,603.885,0.594741,576.510,601.873,0.755656,1
1,664.665,204.246,0.918837,692.090,259.068,0.875573,646.958,257.168,0.822406,572.624,...,515.737,611.718,0.592743,521.600,603.895,0.596312,576.498,601.908,0.757170,1
2,664.684,206.193,0.931179,693.983,257.119,0.875670,647.023,255.218,0.807673,572.645,...,517.692,611.715,0.589091,521.646,603.913,0.586224,578.445,601.916,0.748488,1
3,666.623,212.076,0.964515,696.038,255.150,0.897790,649.037,253.210,0.800950,574.538,...,515.785,609.764,0.605959,521.614,603.875,0.596971,578.452,601.928,0.742447,1
4,666.646,212.079,0.957779,696.036,255.137,0.896309,649.010,251.283,0.797011,574.496,...,515.775,609.786,0.600176,521.619,603.891,0.589176,578.461,601.957,0.746312,1
5,664.672,202.310,0.920205,693.961,257.206,0.880891,646.965,257.134,0.809241,572.579,...,515.780,611.731,0.597030,521.639,603.914,0.598192,576.503,601.935,0.754936,1
6,664.664,204.211,0.916136,693.975,257.200,0.881995,646.959,257.109,0.808656,572.629,...,515.761,611.728,0.594618,521.623,603.916,0.593185,576.504,601.939,0.753448,1
7,664.700,204.222,0.916451,692.098,259.148,0.875034,646.992,257.207,0.835741,572.560,...,515.729,611.715,0.606050,521.586,603.906,0.595842,576.530,601.891,0.744873,1
8,666.639,212.074,0.962055,696.061,255.133,0.898260,649.029,253.176,0.799400,574.501,...,517.704,607.883,0.605212,521.634,603.866,0.599607,578.454,601.934,0.749427,1
9,666.611,210.213,0.968976,696.016,257.061,0.913818,649.043,255.134,0.821040,572.653,...,517.708,609.806,0.610272,521.658,603.904,0.604013,580.384,601.896,0.746584,1
